In [1]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import getpass  # to get the password without showing the input
password = getpass.getpass()

········


In [2]:
# EXERCISE: Create a Python connection with SQL database and retrieve the results of the last two queries as dataframes

connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)

In [3]:
# EXERCISE: Check the number of rentals for each customer for May

df_may = pd.read_sql_query(
    'SELECT customer_id, count(rental_id) as rentals FROM rentals_may GROUP BY customer_id ORDER BY count(rental_id) DESC;',
    engine)
df_may.set_index('customer_id', inplace=True)
df_may.rename(columns = {'rentals':'rentals_may'}, inplace = True)
df_may.head()

,rentals_may
customer_id,
197,8
109,7
506,7
19,6
53,6


In [4]:
# EXERCISE: Check the number of rentals for each customer for June

df_june = pd.read_sql_query(
    'SELECT customer_id, count(rental_id) as rentals FROM rentals_june GROUP BY customer_id ORDER BY count(rental_id) DESC;',
    engine)
df_june.set_index('customer_id', inplace=True)
df_june.rename(columns = {'rentals':'rentals_june'}, inplace = True)
df_june.head()

,rentals_june
customer_id,
31,11
454,10
213,9
267,9
295,9


In [5]:
# EXERCISE: Write a function that checks if a customer borrowed more or less films in the month of June as compared to May.

def june_or_may(customer_id):
    password = getpass.getpass()
    connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
    engine = create_engine(connection_string)
    
    # Check the number of rentals for each customer for May
    df_may = pd.read_sql_query(
        'SELECT customer_id, count(rental_id) as rentals FROM rentals_may GROUP BY customer_id ORDER BY count(rental_id) DESC;',
        engine)
    df_may.set_index('customer_id', inplace=True)
    df_may.rename(columns = {'rentals':'rentals_may'}, inplace = True)
    
    # Check the number of rentals for each customer for June
    df_june = pd.read_sql_query(
        'SELECT customer_id, count(rental_id) as rentals FROM rentals_june GROUP BY customer_id ORDER BY count(rental_id) DESC;',
        engine)
    df_june.set_index('customer_id', inplace=True)
    df_june.rename(columns = {'rentals':'rentals_june'}, inplace = True)
    
    # Join dataframes, fill NaN and sort by customer_id
    rentals = df_may.join(df_june, how="outer").fillna(0).sort_values('customer_id')
    
    if customer_id in rentals.index:
        if rentals.at[customer_id, 'rentals_may'] > rentals.at[customer_id, 'rentals_june']:
            print("Customer ID", customer_id, "rented more films in MAY than in June.")
        elif rentals.at[customer_id, 'rentals_may'] < rentals.at[customer_id, 'rentals_june']:
            print("Customer ID", customer_id, "rented more films in JUNE than in May.")
        else:
            print("Customer ID", customer_id, "rented the same amount of films in May and June.")
    else:
        print("Customer ID", customer_id, "did not rent any films between May and June or is not registered in this database")

In [6]:
# Let's check the function

df_may.join(df_june, how="outer").fillna(0).sort_values('customer_id').head(10)

,rentals_may,rentals_june
customer_id,,
1,2.0,7.0
2,1.0,1.0
3,2.0,4.0
4,0.0,6.0
5,3.0,5.0
6,3.0,4.0
7,5.0,5.0
8,1.0,3.0
9,3.0,2.0


In [7]:
june_or_may(1)

········
Customer ID 1 rented more films in JUNE than in May.


In [8]:
june_or_may(2)

········
Customer ID 2 rented the same amount of films in May and June.


In [9]:
june_or_may(9)

········
Customer ID 9 rented more films in MAY than in June.


In [10]:
june_or_may(1234)

········
Customer ID 1234 did not rent any films between May and June or is not registered in this database
